# Electoral Model on Inverse @PietroMonticone2 

## Abstract

1. Import Twitter graph data 
2. Reverse edge directionality
3. Populate each node with agents with the following attributes 
    * ID $\in \mathbb{N}$
    * position  $\in \mathbb{N}$
    * candidacy $\in \mathbb{B}$
    * candidate score $\in \mathbb{R}$
    * representative score $\in \mathbb{R}$
    * community $\in C \subset \mathbb{N}$
4. Representation Maps / Voting Rules
    * Voter trust flow: each voter weights its out-edges and updates the candidate score of candidates.
    * Candidate trust flow: each candidate weights its out-edges and updates the representative score of candidates.

## Packages

In [5]:
# Data Management 
using DataFrames, DataFramesMeta
using DrWatson

# Statistics
using Random
using Distributions
using StatsBase

# Graphs 
using LightGraphs, SimpleWeightedGraphs, SimpleHypergraphs, GraphIO
using GraphPlot

# Modelling
using Agents
include("./Modules/ElectoralPhysics.jl")

# Data Visualization
using Plots
using AgentsPlots
using PlotThemes

# Python
##using Pkg
##ENV["PYTHON"] = "path/to/python"
##Pkg.build("PyCall")
using PyCall
using PyPlot
nx = pyimport("networkx")
np = pyimport("numpy")
nw = pyimport("netwulf");

   Building Conda ─→ `~/.julia/packages/Conda/3rPhK/deps/build.log`
   Building PyCall → `~/.julia/packages/PyCall/zqDXB/deps/build.log`


## Graph Data Collection

In [10]:
𝒫 = nx.read_graphml("@PietroMonticone2.graphml")
𝒫 = ElectoralPhysics.NetworkX_to_LightGraphs(𝒫)

{472504, 1075454} directed simple Int64 graph with Float64 weights

## Agent Type 

In [11]:
# Voter Definition
mutable struct Voter <: AbstractAgent
    id::Int64             # ∈ ℕ
    pos::Int64            # ∈ ℕ
    candidacy::Bool       # ∈ 𝔹
    score::Float64        # ∈ 𝔹
    final_score::Float64  # \in \bbR
    community::Int64      # ∈ {1,2,3, ..., n}
end

## Parameters

In [14]:
indegrees = indegree(𝒫);
M = length([elem for elem in indegrees if elem != 0]);

InterruptException: InterruptException:

## Model

### Voter  

In [15]:
# Model Initialization
function init_model_voters(𝒫::SimpleWeightedDiGraph{Int64,Float64})
       
    #properties = @dict(N, M, K)
    
    # Create Directed Weighted Graph
    space = GraphSpace(𝒫) 
    model_voters = ABM(Voter, space) #; properties = properties)
    
    N = nv(𝒫)
    
    for id in 1:N
        pos = id
        score = 1.0
        final_score = 0
        if 
            candidacy = true
            add_agent!(pos, model_voters, candidacy, score, final_score)
        else
            candidacy = false
            add_agent!(pos, model_voters, candidacy, score, final_score)
        end
    end
    return model_voters
end;

LoadError: syntax: missing condition in "if" at none:16

### Candidate

In [16]:
# Model Initialization 
function init_model_candidates(model_voters::AgentBasedModel{Voter,GraphSpace{SimpleWeightedDiGraph{Int64,Float64}},typeof(fastest),Nothing}, M::Int64)
    
    properties = @dict(M)
    
    candidates = []
    i=0
    for row in eachrow(sort!(DataFrame(allagents(model_voters_twitter)), :score, rev = true))
        if i == M
            break
        end
        append!(candidates, row[1])
        i+=1
    end
    
    # Induce Subgraph
    graph, vertex_map = induced_subgraph(model_voters.space.graph, [candidate for candidate in candidates])
    space = GraphSpace(graph)
    model_candidates = ABM(Voter, space; properties = properties)
    
    for candidate in allagents(model_voters)
        if candidate.candidacy == true
            add_agent!(candidate.pos, model_candidates, candidate.candidacy, candidate.score, candidate.final_score, candidate.community)
        end
    end
    
    for agent in allagents(model_candidates)
    agent.id = agent.pos
    end
    
    return model_candidates
end;

## Functions 

In [17]:
function vote!(agent, model)
    outneighbors = node_neighbors(agent, model; neighbor_type=:in)
    nvotes = length(outneighbors)
    for outneighbor in outneighbors
        agent_outneighbor = get_node_agents(outneighbor, model)[1]
        add_edge!(model.space.graph, agent.pos, agent_outneighbor.pos, 1/nvotes)
    end
end

function update_score_flow!(model)
    for agent in allagents(model)
        if agent.candidacy == true
            for inneighbor in node_neighbors(agent, model; neighbor_type=:in)
                agent_inneighbor = get_node_agents(inneighbor, model)[1]
                if agent_inneighbor.candidacy == false
                    agent.score = agent.score + agent_inneighbor.score * LightGraphs.weights(model.space.graph)[agent_inneighbor.pos, agent.pos]
                end
            end
        end
    end
end

function update_score_candidates!(model)
    for agent in allagents(model)
        for inneighbor in node_neighbors(agent, model; neighbor_type=:in)
            agent_inneighbor = get_node_agents(inneighbor, model)[1]
            agent.final_score = agent.score + agent_inneighbor.score * LightGraphs.weights(model.space.graph)[agent_inneighbor.pos, agent.pos]
        end
    end
end

function agent_step!(agent, model)
   vote!(agent, model) 
end;

## Simulation 
### Voters

In [ ]:
# Seed Selection
Random.seed!(1234);

# Model Instantiation 
model_voters = init_model_voters(𝒫, indegrees)

data_voters = run!(model_voters, agent_step!, 1);

update_score_flow!(model_voters)

sort!(DataFrame(allagents(model_voters)), :score, rev = true)

### Candidates

In [ ]:
# Model Instantiation 
model_candidates = init_model_candidates(model_voters, M)

# Data Collection
#data_candidates = run!(model_candidates, agent_step!, nsteps);

update_score_candidates!(model_candidates)

sort!(DataFrame(allagents(model_candidates)), :final_score, rev = true)

## Export Election Graph
### Voters

In [ ]:
G = ElectoralPhysics.LightGraphs_to_NetworkX(model_voters.space.graph);
G = nx.convert_node_labels_to_integers(G, first_label=1);
nx.write_graphml(G, "twitter_election_voters.graphml")

### Candidates

In [ ]:
G = ElectoralPhysics.LightGraphs_to_NetworkX(model_candidates.space.graph);
G = nx.convert_node_labels_to_integers(G, first_label=1);
nx.write_graphml(G, "twitter_election_candidates.graphml")

#### Electoral Graph Visualization 

```python 
nw.visualize(G)
```

![](../twitter_elections_candidates.png)